### 供给数据
- 对于小型数据，直接读入内存，而大型数据通过‘输入流水线’的方式读取

### 输入流水线
- 创建文件名列表
- 创建文件名队列
- 创建Reader和Decoder
- 创建样例队列

![](imgs/5.gif)

#### 1 文件名列表
- 使用python的列表
```python
['file1.csv', 'file2.csv', ]
```
- 使用tf.train.match_filenames_once
    - 会在流图中创建一个获取文件名列表的操作(operation)，文件名了列表会在全局初始化时一起被初始化
![](imgs/6.png)

#### 2 文件名队列
- 使用string_input_producer来生成一个先入先出的队列(FIFO)， 文件阅读器会需要它来读取数据
    - **通过设置num_epochs(最大训练周期),shuffle,等进行调节**

#### 3 文件格式与对应的Reader, Decoder
- Reader的read方法进行相应的读取，然后Decoder将其转化为张量形式
- CSV:TextLineReader, decode_csv

In [18]:
import tensorflow as tf


filename_queue = tf.train.string_input_producer(['datasets/dat99.csv', ])
reader = tf.TextLineReader(skip_header_lines=1)
key, value = reader.read_up_to(filename_queue, 5)
print(value)
record_defaults = [['B'], ['F'], [1], [1], [1], ['BJ'], [1.], [1], [1.]]
c1, c2, c3, c4, c5, c6, c7, c8, c9 = tf.decode_csv(
    value, record_defaults=record_defaults)
features = tf.stack([c1, c2, c6])
print(features)

Tensor("ReaderReadUpToV2_5:1", shape=(?,), dtype=string)
Tensor("stack_15:0", shape=(3, ?), dtype=string)


- 使用read_up_to()返回列表 (keys, values) ,通过tf.stack()组合需要的数据
    - 如果是reader：每次read的执行都会从文件中读取一行内容
- decode_csv 操作会解析内容并将其转为张量列表。如果输入的参数有缺失，record_default参数可以根据张量的类型来设置默认值

In [28]:
import tensorflow as tf

filename_queue = tf.train.string_input_producer(['datasets/dat99.csv'])

reader = tf.TextLineReader(skip_header_lines=1)
key, value = reader.read(filename_queue)


record_defaults = [['B'], ['F'], [1.], [1.], [1.], ['BJ'], [1.], [1.], [1.]]
c1, c2, c3, c4, c5, c6, c7, c8, c9 = tf.decode_csv(
    value, record_defaults=record_defaults)
features = tf.stack([c1, c2, c6])#维数

with tf.Session() as sess:
    # 协调器
    coord = tf.train.Coordinator()
    threads = tf.train.start_queue_runners(coord=coord)
    
    for i in range(1000):
        example, label = sess.run([features, c5])
        print(example, label)
    coord.request_stop()
    coord.join(threads)

[b'B' b'F' b'BJ'] 1.0
[b'B' b'F' b'SH'] 1.0
[b'B' b'F' b'SZ'] 1.0
[b'B' b'F' b'CQ'] 1.0
[b'B' b'F' b'BJ'] 1.0
[b'B' b'F' b'SH'] 1.0
[b'B' b'F' b'SZ'] 1.0
[b'B' b'F' b'TJ'] 1.0
[b'B' b'F' b'CQ'] 1.0
[b'B' b'F' b'BJ'] 1.0
[b'B' b'F' b'SH'] 1.0
[b'B' b'F' b'SZ'] 1.0
[b'B' b'F' b'TJ'] 1.0
[b'B' b'F' b'CQ'] 1.0
[b'B' b'F' b'BJ'] 2.0
[b'B' b'F' b'SH'] 2.0
[b'B' b'F' b'SZ'] 2.0
[b'B' b'F' b'TJ'] 2.0
[b'B' b'F' b'CQ'] 2.0
[b'B' b'F' b'BJ'] 2.0
[b'B' b'F' b'SH'] 2.0
[b'B' b'F' b'SZ'] 2.0
[b'B' b'F' b'TJ'] 2.0
[b'B' b'F' b'CQ'] 2.0
[b'B' b'F' b'BJ'] 2.0
[b'B' b'F' b'SH'] 2.0
[b'B' b'F' b'SZ'] 2.0
[b'B' b'F' b'TJ'] 2.0
[b'B' b'F' b'CQ'] 2.0
[b'B' b'F' b'BJ'] 2.0
[b'B' b'F' b'SH'] 2.0
[b'B' b'F' b'SZ'] 2.0
[b'B' b'F' b'TJ'] 2.0
[b'B' b'F' b'CQ'] 2.0
[b'B' b'F' b'BJ'] 2.0
[b'B' b'F' b'SH'] 2.0
[b'B' b'F' b'SZ'] 2.0
[b'B' b'F' b'TJ'] 2.0
[b'B' b'F' b'CQ'] 2.0
[b'B' b'F' b'BJ'] 3.0
[b'B' b'F' b'SH'] 3.0
[b'B' b'F' b'SZ'] 3.0
[b'B' b'F' b'TJ'] 3.0
[b'B' b'F' b'CQ'] 3.0
[b'B' b'F' b'BJ'] 3.0
[b'B' b'F'

[b'B' b'M' b'BJ'] 5.0
[b'B' b'M' b'SH'] 5.0
[b'B' b'M' b'SZ'] 5.0
[b'B' b'M' b'TJ'] 5.0
[b'B' b'M' b'CQ'] 5.0
[b'B' b'M' b'BJ'] 5.0
[b'B' b'M' b'SH'] 5.0
[b'B' b'M' b'SZ'] 5.0
[b'B' b'M' b'TJ'] 5.0
[b'B' b'M' b'CQ'] 5.0
[b'B' b'M' b'BJ'] 6.0
[b'B' b'M' b'SH'] 6.0
[b'B' b'M' b'SZ'] 6.0
[b'B' b'M' b'TJ'] 6.0
[b'B' b'M' b'CQ'] 6.0
[b'B' b'M' b'BJ'] 6.0
[b'B' b'M' b'SH'] 6.0
[b'B' b'M' b'SZ'] 6.0
[b'B' b'M' b'TJ'] 6.0
[b'B' b'M' b'CQ'] 6.0
[b'B' b'M' b'BJ'] 6.0
[b'B' b'M' b'SH'] 6.0
[b'B' b'M' b'SZ'] 6.0
[b'B' b'M' b'TJ'] 6.0
[b'B' b'M' b'CQ'] 6.0
[b'B' b'M' b'BJ'] 6.0
[b'B' b'M' b'SH'] 6.0
[b'B' b'M' b'SZ'] 6.0
[b'B' b'M' b'TJ'] 6.0
[b'B' b'M' b'CQ'] 6.0
[b'B' b'M' b'BJ'] 6.0
[b'B' b'M' b'SH'] 6.0
[b'B' b'M' b'SZ'] 6.0
[b'B' b'M' b'TJ'] 6.0
[b'B' b'M' b'CQ'] 6.0
[b'B' b'M' b'BJ'] 6.0
[b'B' b'M' b'SH'] 6.0
[b'B' b'M' b'SZ'] 6.0
[b'B' b'M' b'TJ'] 6.0
[b'B' b'M' b'CQ'] 6.0
[b'B' b'M' b'BJ'] 6.0
[b'B' b'M' b'SH'] 6.0
[b'B' b'M' b'SZ'] 6.0
[b'B' b'M' b'TJ'] 6.0
[b'B' b'M' b'CQ'] 6.0
[b'B' b'M'

- 在调用run或者eval去执行read之前， 你必须调用tf.train.start_queue_runners来将文件名填充到队列。否则read操作会被阻塞到文件名队列中有值为止
- 注意一些csv文件有标题，需要去掉，可以参考文档的Reader类
- 速度很快

- 标准TF格式(protocol buffer)，TFRecord
    - TFRecoder数据类型是通过tf.train.Example 格式存储的，这种格式包含了从属性名称到取值的字典，其中名称为字符串类型，取值可以是ByteList,Int64List,FloatList，结构声明在P86

- 二进制
    - 从二进制文件中读取固定长度纪录， 可以使用tf.FixedLengthRecordReader的tf.decode_raw操作。decode_raw操作可以讲一个字符串转换为一个uint8的张量。

    - 举例来说，the CIFAR-10 dataset的文件格式定义是：每条记录的长度都是固定的，一个字节的标签，后面是3072字节的图像数据。uint8的张量的标准操作就可以从中获取图像片并且根据需要进行重组。 例子代码可以在tensorflow/models/image/cifar10/cifar10_input.py找到，具体讲述可参见教程.

#### 4 创建样例队列
- 经过上面的步骤得到的是张量
- 通过调用tf.train.start_queue_runners()启动线程进行读取，这里还需要一个协调器，不然会出现线程不协调导致的越界之类的错误，同时，**启动协调器必须执行tf.local_variables_initializer()对其进行初始化.参考一下上面的程序，没有进行初始化，但是并没有出错，不过书中的程序不同，在具体使用时再参考**，书上的使用如下：

In [ ]:
# Create the graph, etc.
init_op = tf.initialize_all_variables()
'''或者聚合初始化(P85)
init_op = tf.group(tf.global_variables_initializer(),
                    tf.local_variables_initializer())#local的是使用协调器要求的
'''
# Create a session for running operations in the Graph.
sess = tf.Session()

# Initialize the variables (like the epoch counter).
sess.run(init_op)


# Start input enqueue threads.
coord = tf.train.Coordinator()
threads = tf.train.start_queue_runners(sess=sess, coord=coord)
#相比直接使用，加入出错判断，这种应该会更加适合实际使用
try:
    while not coord.should_stop:
        #run training steps or whatever
        sess.run(train_op)
except tf.errors.OutOfRangeError:
    print('Done training -- epoch limit reached')
finally:
    # When done, ask the threads to stop.
    coord.request_stop()
    
# Wait for threads to finish.
coord.join(threads)
sess.close()

#### 以下给出两个来自别的书上TFRecord数据类型的例子

In [3]:
#mnist转化为TFRecord
import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data
import numpy as np

def _int64_feature(value):
    return tf.train.Feature(int64_list=tf.train.Int64List(value=[value]))
def _bytes_feature(value):
    return tf.train.Feature(bytes_list=tf.train.BytesList(value=[value]))

mnist = input_data.read_data_sets("datasets/mnist_data", dtype=tf.uint8, one_hot=True)
images = mnist.train.images
labels = mnist.train.labels
pixels = images.shape[1]
num_examples = mnist.train.num_examples

filename = "datasets/mnist.tfrecords"
#创建writer用于写入
writer = tf.python_io.TFRecordWriter(filename)
for index in range(num_examples):
    image_row = images[index].tostring()#这里将image编码成byte格式
    example = tf.train.Example(features=tf.train.Features(feature={
        'pixels': _int64_feature(pixels),
        'label': _int64_feature(np.argmax(labels[index])),
        'image': _bytes_feature(image_row)
    }))
    #写入文件
    writer.write(example.SerializeToString())
writer.close()

URLError: <urlopen error [WinError 10060] 由于连接方在一段时间后没有正确答复或连接的主机没有反应，连接尝试失败。>

In [1]:
#相应的读取
reader = tf.TFRecordReader()
filename_queue = tf.train.string_input_producer(['datasets/mnist.tfrecords', ])
#这里和之前csv读取的例子一样，也可以使用read_up_to()
_, value = reader.read(filename_queue)
#如果是多个样例，这里应该使用parse_example()
features = tf.parse_single_example(
    value,
    features={
        #这里解析方法还有tf.VarLenFeature(),结果是稀疏张量
        'image': tf.FixedLenFeature([], tf.string),#参数是shape，dtype，default_value
        'pixels': tf.FixedLenFeature([], tf.int64),
        'label': tf.FixedLenFeature([], tf.int64)
    })
#tf.decode_raw()可以将字符串解析成图像
image_ = tf.decode_raw(features['image'], tf.uint8)##一定要注意这里的数据类型
label_ = tf.cast(features['label'], tf.int32)
pixels_ = tf.cast(features['pixels'], tf.int32)

with tf.Session() as sess:
    coord = tf.train.Coordinator()
    threads = tf.train.start_queue_runners(coord=coord)
    for i in range(10):
        image, label, pixels = sess.run([image_, label_, pixels_])
    coord.request_stop()
    coord.join(threads)

NameError: name 'tf' is not defined

#### 另一个例子

In [ ]:
#生成数据
import tensorflow as tf

#TFRecoder文件的帮助函数
def _int64_feature(values):
    return tf.train.Feature(int64_list=tf.train.Int64List(value=[value]))

#将数据写入不同文件
num_shareds = 2 #文件数
instance_per_shard = 2 #文件有多少数据

for i in range(num_shareds):
    # 按0000n-of-0000m的后缀区分文件。n代表当前文件编号，m代表文件总数
    filename = ('data/data.tfrecords-%.5d-of-%.5d' % (i, num_shareds))
    writer = tf.python_io.TFRecordWriter(filename)
    
    #数据封装成Example结构，写入TFRecoder文件
    for j in range(instances_per_shard):
        example = tf.train.Example(
            features = tf.train.Features(feature={
                'i':_int64_feature(i),
                'j':_int64_feature(j)
            }))
        writer.write(example.SerializeToString())
    writer.close()

In [ ]:
import tensorflow as tf

# 获取文件列表
files = tf.train.match_filenames_once('data/data.tfrecords-*')

# 创建文件输入队列
filename_queue = tf.train.string_input_producer(files, shuffle=False)#再次注意，这里可以设置训练周期和shuffle

# 读取并解析Example
reader = tf.TFRecordReader()
_, serialized_example = reader.read(filename_queue)
features = tf.parse_single_example(
    serialized_example,
    features={
        'i': tf.FixedLenFeature([], tf.int64),
        'j': tf.FixedLenFeature([], tf.int64)
    })

with tf.Session() as sess:
    # 使用match_filenames_once需要用local_variables_initializer初始化其中的一些变量，不然会出错
    sess.run(
        [tf.global_variables_initializer(),
         tf.local_variables_initializer()])
    '''或者聚合初始化(P85)
    init_op = tf.group(tf.global_variables_initializer(),
                        tf.local_variables_initializer())#local的是使用协调器要求的
    '''
    # 打印文件名
    print(sess.run(files))

    # 用Coordinator协同线程，并启动线程
    coord = tf.train.Coordinator()
    threads = tf.train.start_queue_runners(coord=coord)

    # 获取数据
    for i in range(6):
        print(sess.run([features['i'], features['j']]))
    coord.request_stop()
    coord.join(threads)

#### 5 预处理
- 对输入的样本进行任意的预处理， 这些预处理不依赖于训练参数， 在tensorflow/models/image/cifar10/cifar10.py可以找到数据归一化， 提取随机数据片，增加噪声或失真等等预处理的例子。

### Tips:
- tf.train.match_filenames_once()中模式和linux一样：
    - *:任意多字符  ？：任意字符一次

In [ ]:
#读取队列测试
import tensorflow as tf

directory = "dataset\\data-*.TFRecord"
file_names = tf.train.match_filenames_once(directory)

init = (tf.global_variables_initializer(), tf.local_variables_initializer())

with tf.Session() as sess:
    sess.run(init)
    print(sess.run(file_names))

### 至此，已经实现上面动图中的example Queue，在P87，作者提出实际应用时还需要一个batch Queue

#### 6 批处理

#### 创建
- 在数据输入管线的末端， 我们需要有另一个队列来执行输入样本的训练，评价和推理。因此我们使用tf.train.shuffle_batch 函数来对队列中的样本进行乱序处理(参考文档的用法，以及P86)

In [ ]:
def read_my_file_format(filename_queue):
    """这部分就是前面解决的问题"""
    reader = tf.SomeReader()
    key, record_string = reader.read(filename_queue)
    example, label = some_decoder(record_string)
    #数据预处理
    processed_example = some_processing(example)
    return processed_example, label

def input_pipline(filenames, batch_size, num_epochs=None):
    filename_queue = tf.train.string_input_producer(
        filenames, num_epochs=num_epochs, shuffle=True
    )
    example, label = read_my_file_format(filename_queue)
    #以下为创建batch Queue，注意这里batch Queue类似插入在喂食到网络前的预处理层
    min_after_dequeue = 1024
    capcity = min_after_dequeue + 3*batch_size
    # min_after_dequeue defines how big a buffer we will randomly sample(用于抽样的队列长度)
    #        -- bigger means better shuffling but slower start up and more memory used.
    # capacity 批数据队列容量
    #        Recommendation(推荐):
    #          min_after_dequeue + (num_threads + a small safety margin) * batch_size
    
    example_batch, label_batch = tf.train.shuffle_batch(
        [example, label], batch_size, capcity, min_after_dequeue
    )
    return example_batch, label_batch

- 这里还有一个类似的tf.train.shuffle_batch_join 函数：

In [ ]:
def input_pipeline(filenames, batch_size, read_threads, num_epochs=None):
    filename_queue = tf.train.string_input_producer(
      filenames, num_epochs=num_epochs, shuffle=True
    )
    #使用多线程处理
    example_list = [read_my_file_fromat(filename_queue)
                   for _ in range(read_threads)]
    min_after_dequeue = 1024
    capacity = min_after_dequeue + 3 * batch_size
    example_batch, label_batche = tf.train.shuffle_batch_join(
        example_list, batch_size, capacity, min_after_dequeue
    )
    
    return example_batch, label_batch

- 另一种替代方案是： 使用tf.train.shuffle_batch 函数,设置num_threads的值大于1。 这种方案可以保证同一时刻只在一个文件中进行读取操作(但是读取速度依然优于单线程)，而不是之前的同时读取多个文件。这种方案的优点是：

    - 避免了两个不同的线程从同一个文件中读取同一个样本。
    - 避免了过多的磁盘搜索操作。
- 你一共需要多少个读取线程呢？ 函数tf.train.shuffle_batch*为TensorFlow图提供了获取文件名队列中的元素个数之和的方法。 如果你有足够多的读取线程， 文件名队列中的元素个数之和应该一直是一个略高于0的数。具体可以参考[TensorBoard:可视化学习](http://www.tensorfly.cn/tfdoc/how_tos/summaries_and_tensorboard.html).

### 模型中的具体调用模板

In [ ]:
x_batch = input_pipline()
with tf.Session as sess:
    init_op = tf.group([
        tf.global_variables_initializer(),
        tf.local_variables_initializer()
    ])
    sess.run(init_op)
    #协调器。现在代码有较好的封装
    coord = tf.train.Coordinator()
    threads = tf.train.start_queue_runners(coord=coord)
    #之前写项目直接八batch放进数据流图了。。。
    try:
        for _ in range(1000):
            if not coord.should_stop():
                batch = sess.run(x_batch)
                sess.run(train_op, feed_dict={X: batch})
                print('xx')
    except tf.errors.OutOfRangeError:
        print('OutOfRangeError')
    finally:
        coord.request_stop()
        print('Finish reading')
    coord.join(threads)

### 使用tf.data
- [参考](https://mp.weixin.qq.com/s?__biz=MzUyMjE2MTE0Mw==&mid=2247485929&idx=2&sn=1c76b725a5f941101cfe74d0a4e974a9&chksm=f9d15771cea6de6733c9ebdd6c5198a2ce6317367f596882773af680eaa50fd5e846cfd73559&mpshare=1&scene=23&srcid=0714u4Mpd2mZ53vuFUEzC3Cc#rd)
- 使用tf.data.Dataset表示一个数据集合，集合的每个元素为一个batch
- 使用tf.data.Iterator从数据集中提取数据，它本身是迭代器

#### 创建Dataset

In [ ]:
#从np.array
dataset1 = tf.data.Dataset.from_tensor_slices(np.random.randn((5, 10))
print(dataset1.output_types)# ==> "tf.float32"
print(dataset1.output_shapes)  # ==> "(10,)" 

#从tensor
dataset2 = tf.data.Dataset.from_tensor_slices((
    tf.random_uniform([4]),
    tf.random_uniform([4, 100], maxval=100, dtype=tf.int32)
))
print(dataset2.output_types)  # ==> "(tf.float32, tf.int32)"
print(dataset2.output_shapes)  # ==> "((), (100,))"

# 从文件
filenames = ["/var/data/file1.tfrecord", "/var/data/file2.tfrecord"]
dataset3 = tf.data.TFRecordDataset(filenames)

##### 支持多种链式调用

In [ ]:
dataset = tf.data.TFRecordDataset(filenames)

dataset = dataset.map(...)  # 解析数据或者对数据预处理，如normalize.

dataset = dataset.repeat()  # 重复数据集，一般设置num_epochs

dataset = dataset.batch(32) # 形成batch

#### 创建Iterator
- 迭代器，4种类型

##### One-shot Iterator
- 这是最简单的Iterator，它仅仅遍历整个数据集一次，而且不需要显示初始化，下面是个实例：

In [ ]:
dataset = tf.data.Dataset.from_tensor_slices(np.arange(10))
iterator = dataset.make_one_shot_iterator()
next_element = iterator.get_next()
with tf.Session() as sess:
    for i in range(10):
        sess.run(next_element) # 0, 1, ..., 9

##### Initializable Iterator
- 需要初始化，可以在每次初始化时传入不同参数

In [ ]:
max_value = tf.placeholder(tf.int64, [])
dataset = tf.data.Dataset.range(max_value)
iterator = dataset.make_initializable_iterator()
next_element = iterator.get_next()
with tf.Session() as sess:
    sess.run(iterator.initializer, feed_dict={max_value: 10})
    for i in range(10):
        print(sess.run(next_element)) # 0, 1, ..., 9

##### Reinitializable Iterator
- 相比initializable Iterator，它可以支持从不同的Dataset进行初始化，有时候你需要训练集和测试集，但是两者并不同，此时就可以定义两个不同的Dataset，并配合reinitializable Iterator来定义一个通用的迭代器，在使用前只需要送入不同的Dataset进行初始化就可以，

In [ ]:
train_data = np.random.randn(100, 5)
test_data = np.random.randn(20, 5)
train_dataset = tf.data.Dataset.from_tensor_slices(train_data)
test_dataset = tf.data.Dataset.from_tensor_slices(test_data)

# 创建一个reinitializable iterator
re_iterator = tf.data.Iterator.from_structure(train_dataset.output_types,
                                              train_dataset.output_shapes)
next_element = re_iterator.get_next()
train_init_op = re_iterator.make_initializer(train_dataset)
test_init_op = re_iterator.make_initializer(test_dataset)
with tf.Session() as sess:    # 训练
    n_epochs = 2
    for i in range(n_epochs):
        sess.run(train_init_op)
        for j in range(100):
            print(sess.run(next_element))
    # 测试
    sess.run(test_init_op)
    for i in range(20):
        print(sess.run(next_element))

##### Feedable Iterator
- 对于reinitializable iterator，它可以支持送入不同Dataset，从而完成数据集的切换，但是每次切换时必须要重新初始化。对于Feedable Iterator，其可以认为支持送入不同的Iterator，通过切换迭代器的string_handle来完成不同数据集的切换，并且在切换时迭代器的状态还会被保留，**这相比reinitializable iterator更加灵活**，下面是一个实例：

In [ ]:
train_data = np.random.randn(100, 5)
val_data = np.random.randn(20, 5)
n_epochs = 20train_dataset = tf.data.Dataset.from_tensor_slices(train_data).repeat(n_epochs)
val_dataset = tf.data.Dataset.from_tensor_slices(val_data)

# 创建一个feedable iterator
handle = tf.placeholder(tf.string, [])
feed_iterator = tf.data.Iterator.from_string_handle(handle, train_dataset.output_types,
                                                  train_dataset.output_shapes)
next_element = feed_iterator.get_next()

# 创建不同的iterator
train_iterator = train_dataset.make_one_shot_iterator()
val_iterator = val_dataset.make_initializable_iterator()
with tf.Session() as sess:
    # 生成对应的handle
    train_handle = sess.run(train_iterator.string_handle())
    val_handle = sess.run(val_iterator.string_handle())
    # 训练
    for n in range(n_epochs):
        for i in range(100):
            print(i, sess.run(next_element, feed_dict={handle: train_handle}))
                # 验证
        if n % 10 == 0:
            sess.run(val_iterator.initializer)
                    for i in range(20):
                print(sess.run(next_element, feed_dict={handle: val_handle}))

### 问题：（参见官方文档）
-  在达到最大训练迭代数的时候如何清理关闭线程?
- 筛选记录或产生每个记录的多个样本
- 稀疏输入数据

### 预加载数据
- 这仅用于可以完全加载到存储器中的小的数据集。有两种方法：

    - 存储在常数中。
    - 存储在变量中，初始化后，永远不要改变它的值。
- 使用常数更简单一些，但是会使用更多的内存（因为常数会内联的存储在数据流图数据结构中，这个结构体可能会被复制几次）。

In [ ]:
#常量方法
training_data = ...
training_labels = ...
with tf.Session():
    input_data = tf.constant(training_data)
    input_labels = tf.constant(training_labels)
    ...

In [ ]:
#变量方法
training_data = ...
training_labels = ...
with tf.Session() as sess:
    data_initializer = tf.placeholder(dtype=training_data.dtype,
                                    shape=training_data.shape)
    label_initializer = tf.placeholder(dtype=training_labels.dtype,
                                     shape=training_labels.shape)
    input_data = tf.Variable(data_initalizer, trainable=False, collections=[])
    input_labels = tf.Variable(label_initalizer, trainable=False, collections=[])
    ...
    sess.run(input_data.initializer,
           feed_dict={data_initializer: training_data})
    sess.run(input_labels.initializer,
           feed_dict={label_initializer: training_lables})

- 设定trainable=False 可以防止该变量被数据流图的 GraphKeys.TRAINABLE_VARIABLES 收集, 这样我们就不会在训练的时候尝试更新它的值； 设定 collections=[] 可以防止GraphKeys.VARIABLES 收集后做为保存和恢复的中断点。

- 无论哪种方式，[tf.train.slice_input_producer function](http://www.tensorfly.cn/tfdoc/api_docs/python/io_ops.html#slice_input_producer)函数可以被用来每次产生一个切片。这样就会让样本在整个迭代中被打乱，所以在使用批处理的时候不需要再次打乱样本。所以我们不使用shuffle_batch函数，取而代之的是纯[tf.train.batch](http://www.tensorfly.cn/tfdoc/api_docs/python/io_ops.html#batch) 函数。 如果要使用多个线程进行预处理，需要将num_threads参数设置为大于1的数字。
#### 例子
- 在tensorflow/g3doc/how_tos/reading_data/fully_connected_preloaded.py 中可以找到一个MNIST例子，使用常数来预加载。 另外使用变量来预加载的例子
- 在tensorflow/g3doc/how_tos/reading_data/fully_connected_preloaded_var.py，你可以用上面 fully_connected_feed 和 fully_connected_reader 的描述来进行比较。

### 多输入通道
- 通常你会在一个数据集上面训练，然后在另外一个数据集上做评估计算(或称为 "eval")。 这样做的一种方法是，实际上包含两个独立的进程：

    - 训练过程中读取输入数据，并定期将所有的训练的变量写入还原点文件）。
    - 在计算过程中恢复还原点文件到一个推理模型中，读取有效的输入数据。
- 这两个进程在下面的例子中已经完成了：the example CIFAR-10 model，有以下几个好处：

    - eval被当做训练后变量的一个简单映射。
    - 你甚至可以在训练完成和退出后执行eval。
- 您可以在同一个进程的相同的数据流图中有训练和eval，并分享他们的训练后的变量。参考the shared variables tutorial.